# Importing the dependencies

In [11]:
%pip install nltk
%pip install --upgrade nltk

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [13]:
import random 
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM , Dense,Activation
from tensorflow.keras.optimizers import RMSprop

In [14]:
text_df=pd.read_csv('fake_or_real_news.csv')

In [16]:
text_df

id                                              title  \
0      8476                       You Can Smell Hillary’s Fear   
1     10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2      3608        Kerry to go to Paris in gesture of sympathy   
3     10142  Bernie supporters on Twitter erupt in anger ag...   
4       875   The Battle of New York: Why This Primary Matters   
...     ...                                                ...   
6330   4490  State Department says it can't find emails fro...   
6331   8062  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
6332   8622  Anti-Trump Protesters Are Tools of the Oligarc...   
6333   4021  In Ethiopia, Obama seeks progress on peace, se...   
6334   4330  Jeb Bush Is Suddenly Attacking Trump. Here's W...   

                                                   text label  
0     Daniel Greenfield, a Shillman Journalism Fello...  FAKE  
1     Google Pinterest Digg Linkedin Reddit Stumbleu...  FAKE  
2     U.S. Secretary of State John F. Kerry said Mon...  REAL  
3     — Kaydee King (@KaydeeKing) November 9, 2016 T...  FAKE  
4     It's primary day in New York and front-runners...  REAL  
...                                                 ...   ...  
6330  The State Department told the Republican Natio...  REAL  
6331  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...  FAKE  
6332   Anti-Trump Protesters Are Tools of the Oligar...  FAKE  
6333  ADDIS ABABA, Ethiopia —President Obama convene...  REAL  
6334  Jeb Bush Is Suddenly Attacking Trump. Here's W...  REAL  

[6335 rows x 4 columns]

In [18]:
# took the text from the above text above and joined them to make a paragraph of text
# which is the joined_text
text=list(text_df.text.values)
joined_text= " ".join(text)

In [74]:
partial_text=joined_text[:100000]
tokenizer=RegexpTokenizer(r'\w+')
tokens =tokenizer.tokenize(partial_text.lower())

In [75]:
# tokens
unique_tokens=np.unique(tokens)
unique_token_index={token:idx for idx, token in enumerate(unique_tokens)}


In [76]:
unique_tokens

array(['0', '000', '1', ..., 'zarif', 'zero', 'zhou'], dtype='<U18')

In [77]:
n_words=10
input_words=[]
next_words=[]


for i in range(len(tokens)-n_words):
    input_words.append(tokens[i:i+n_words])
    next_words.append(tokens[i+n_words])

In [78]:
x=np.zeros((len(input_words),n_words,len(unique_tokens)),dtype=bool)
y=np.zeros((len(next_words),len(unique_tokens)),dtype=bool)

In [79]:
for i,words in enumerate(input_words):
    for j,word in enumerate(words):
        x[i,j,unique_token_index[word]]=1
    y[i,unique_token_index[next_words[i]]]=1

In [80]:
# TRAINING THE MODEL
model=Sequential()
model.add(LSTM(128,input_shape=(n_words,len(unique_tokens)),return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation('softmax'))

In [81]:
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(learning_rate=0.01), metrics=['accuracy'])
model.fit(x,y,batch_size=128,epochs=40,shuffle=True)

Epoch 1/40
135/135 [==============================] - 32s 204ms/step - loss: 6.8887 - accuracy: 0.0546
Epoch 2/40
135/135 [==============================] - 27s 202ms/step - loss: 6.6630 - accuracy: 0.0613
Epoch 3/40
135/135 [==============================] - 27s 200ms/step - loss: 6.5265 - accuracy: 0.0667
Epoch 4/40
135/135 [==============================] - 27s 200ms/step - loss: 6.3665 - accuracy: 0.0753
Epoch 5/40
135/135 [==============================] - 26s 196ms/step - loss: 6.1669 - accuracy: 0.0854
Epoch 6/40
135/135 [==============================] - 26s 195ms/step - loss: 5.9522 - accuracy: 0.0977
Epoch 7/40
135/135 [==============================] - 28s 206ms/step - loss: 5.7142 - accuracy: 0.1153
Epoch 8/40
135/135 [==============================] - 27s 199ms/step - loss: 5.4575 - accuracy: 0.1333
Epoch 9/40
135/135 [==============================] - 25s 182ms/step - loss: 5.1856 - accuracy: 0.1578
Epoch 10/40
135/135 [==============================] - 24s 175ms/step - l

In [82]:
model.save('my_model.h5')

c:\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [83]:
model=load_model('my_model.h5')

In [84]:
def predict_next_word (input_text,n_best):
    input_text=input_text.lower()
    x=np.zeros((1,n_words,len(unique_tokens)))
    for i,word in enumerate(input_text.split()):
        x[0,i,unique_token_index[word]]=1

    predicitions=model.predict(x)[0]
    return np.argpartition(predicitions,-n_best)[-n_best:]

In [85]:
possible=predict_next_word("He will have to look into this thing and he",5)

1/1 [==============================] - 2s 2s/step


In [86]:
print([unique_tokens[idx] for idx in possible])

['once', 'until', 'at', 'when', 'through']


In [87]:
def generate_text(input_text,text_length,creativity=3):
    word_sequence=input_text.split()
    current=0
    for _ in range(text_length):
        sub_sequence=" ".join(tokenizer.tokenize(' '.join(word_sequence).lower())[current:current+n_words])
        try:
            choice=unique_tokens[random.choice(predict_next_word(sub_sequence,creativity))]
        except:
            choice=random.choice(unique_tokens)
        word_sequence.append(choice)
        current+=1
    return " ".join(word_sequence)

In [88]:
generate_text("He will have to look into this thing and he",50,5)

1/1 [==============================] - 0s 57ms/step


1/1 [==============================] - 0s 49ms/step


'He will have to look into this thing and he until just often a key ayatollah event votes and questions may force his childhood same odious removal path we turn that our know it to do this w unscathed that first two them for a win all majority and has might obama much about this results not an campaign s'